# Extracting GLDAS-2.0

---

should have: 
<br>earthengine-api
<br>geemap

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import time
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

In [2]:
import ee

ee.Initialize()

### Initialization
Initialization of the area, the point, the time period.
<br> Please change the filterdate parameters as necessary.

In [3]:
aoi = ee.Geometry.Point(-62.119581801721324,-10.215769049745338)
gldas = (ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')
         .filterDate(datetime.datetime(2020, 1, 15), datetime.datetime(2020, 1, 16))
         .filterBounds(aoi))


# gldas = gldas.filterDate(datetime.datetime(2000, 1, 1), datetime.datetime(2020, 1, 16))


#### Selecting bands / defining their names in-program

I put all the bands as they are, according to [Earth Explorer](https://explorer.earthengine.google.com/#detail/NASA%2FGLDAS%2FV20%2FNOAH%2FG025%2FT3H).

However, it seems that not all data is used in [Mr. Effrain's code.](https://code.earthengine.google.com/c54330f0b4cbcc19cc24a98f9ea0b531) The excess bands have been commented out.


In [4]:
Albedo_inst = gldas.select('Albedo_inst') 
Lwnet_tavg = gldas.select('Lwnet_tavg') 
Psurf_f_inst = gldas.select('Psurf_f_inst')
Qair_f_inst = gldas.select('Qair_f_inst')
Qh_tavg = gldas.select('Qh_tavg')
Qle_tavg  = gldas.select('Qle_tavg')
Rainf_f_tavg = gldas.select('Rainf_f_tavg')
RootMoist_inst = gldas.select('RootMoist_inst')
SWdown_f_tavg = gldas.select('SWdown_f_tavg')
Swnet_tavg = gldas.select('Swnet_tavg')
Tair_f_inst = gldas.select('Tair_f_inst')
Wind_f_inst = gldas.select('Wind_f_inst')

In [5]:
# # shortwave radiation at the ground surface
# SWdown_f_tavg - downward short-wave radiation flux
# SWnet_tavg - net short wave radiation flux
# # wind
# Wind_f_inst - wind speed
# # precipitation
# Rainf_f_tavg - Total precipitation rate
# # temperature
# Tair_f_inst - Air temperature
# # Vapor pressure
# Psurf_f_inst - Pressure
# # radiation
# Qh_tavg - sensible heat net flux
# Qle_tavg - latent heat flux
# Albedo_inst - albedo (reflected radiation)
# Lwnet_tavg - longwave radiation flux


# #other
# Qair_f_inst - specific humidity
# RootMoist_inst = root moisture

In [6]:
# AvgSurfT_inst = gldas.select('AvgSurfT_inst')
# CanopInt_inst = gldas.select('CanopInt_inst')
# ECanop_tavg = gldas.select('ECanop_tavg')
# ESoil_tavg = gldas.select('ESoil_tavg')
# Evap_tavg = gldas.select('Evap_tavg')
# LWdown_f_tavg = gldas.select('LWdown_f_tavg')
# PotEvap_tavg = gldas.select('PotEvap_tavg')
# Qg_tavg = gldas.select('Qg_tavg')
# Qs_acc = gldas.select('Qs_acc')
# Qsb_acc = gldas.select('Qsb_acc')
# Qsm_acc = gldas.select('Qsm_acc')
# Rainf_tavg = gldas.select('Rainf_tavg')
# SWE_inst = gldas.select('SWE_inst')
# SnowDepth_inst = gldas.select('SnowDepth_inst')
# Snowf_tavg = gldas.select('Snowf_tavg')
# Snowf_tavg = gldas.select('Snowf_tavg')
# SoilMoi0_10cm_inst = gldas.select('SoilMoi0_10cm_inst')
# SoilMoi10_40cm_inst = gldas.select('SoilMoi10_40cm_inst')
# SoilMoi40_100cm_inst = gldas.select('SoilMoi40_100cm_inst')
# SoilMoi100_200cm_inst = gldas.select('SoilMoi100_200cm_inst')
# SoilTMP0_10cm_inst = gldas.select('SoilTMP0_10cm_inst')
# SoilTMP10_40cm_inst = gldas.select('SoilTMP10_40cm_inst')
# SoilTMP40_100cm_inst = gldas.select('SoilTMP40_100cm_inst')
# SoilTMP100_200cm_inst = gldas.select('SoilTMP100_200cm_inst')
# Tveg_tavg = gldas.select('Tveg_tavg')


#### Main Algorithm (WIP)
addTime function adds a column in csv with the date in years since the epoch. (I could not figure out how to get feature and image data integrated yet.) 
<br>
I keep getting the error "bad date/time" when attempting to use ee.Date to set the image properties.
<br>
The rest of the function reduces and finds the mean of the data.
<br>
It is then exported.

In [7]:
# def addtime(image):
#     return image.addBands(image.metadata('system:time_start').divide(1000*60*60*24*365))

# RootMoist_inst = RootMoist_inst.map(addtime)

In [8]:
# def addDate(image):
#     date = ee.Date(image.get('system:index'))
# #     date = date.format("YYYY-mm-dd")
#     return image.set({'Date': date})

# RootMoist_inst = RootMoist_inst.map(addDate)

In [9]:
# def date(image, feature):
#     date = ee.Date(image.get('value'))
#     return feature.set({"Date": date})

In [10]:
# def date(image):
#     date = ee.Image.date(image)
#     return image.set('date', date)
# #     return image.set({'Date': date})

# RootMoist_inst = RootMoist_inst.map(date)
# RootMoist_inst.getInfo()

In [11]:
RootMoist_inst.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'NASA/GLDAS/V021/NOAH/G025/T3H',
 'version': 1590965647019730.0,
 'properties': {'system:visualization_0_min': '250.0',
  'type_name': 'ImageCollection',
  'thumb': 'https://mw1.google.com/ges/dd/images/NASA_GLDAS_thumb.png',
  'description': '<p>Global Land Data Assimilation System (GLDAS) ingests satellite and\nground-based observational data products. Using advanced land surface\nmodeling and data assimilation techniques, it generates optimal\nfields of land surface states and fluxes.</p><p>GLDAS-2.1 is one of two components of the GLDAS Version 2 (GLDAS-2) dataset,\nthe second being GLDAS-2.0. GLDAS-2.1 is analogous to GLDAS-1 product stream,\nwith upgraded models forced by a combination of <a href="https://ldas.gsfc.nasa.gov/gldas/GLDASforcing.php">GDAS, disaggregated GPCP,\nand AGRMET radiation data sets</a>.</p><p>The GLDAS-2.1 simulation started on January 1, 2000 using the\nconditions from the GLDAS-2.0 simulation. This simulati

In [12]:
# def date(feature):
#     date = feature.get('system:index')
#     return feature.set({'Date': date})
# RootMoist_inst = RootMoist_inst.map(date)
# # RootMoist_inst.getInfo()


In [13]:
# def func(collection):   
#     def reduce(image):
#         reduced = image.reduceRegions(collection=aoi, reducer=ee.Reducer.mean(), scale=25000)
#         image = image.set('value', reduced)
#         return reduced
#     red_feature = collection.map(reduce)
#     return red_feature.flatten()

In [14]:
# results = func(RootMoist_inst)
# results.getInfo()

https://stackoverflow.com/questions/55484656/iterating-over-a-gee-featurecollection


In [15]:
def reduce(image):
    reduced = image.reduceRegions(collection=aoi, reducer=ee.Reducer.mean(), scale=25000)
    image = image.set('value', reduced)
    return reduced

In [16]:
results_1 = RootMoist_inst.map(reduce)
results = results_1.flatten()

In [17]:
hh = ee.FeatureCollection(results)
hh.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-62.119581801721324, -10.215769049745338]},
   'id': 'A20200115_0000_0',
   'properties': {'mean': 491.3389892578125}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-62.119581801721324, -10.215769049745338]},
   'id': 'A20200115_0300_0',
   'properties': {'mean': 490.59600830078125}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-62.119581801721324, -10.215769049745338]},
   'id': 'A20200115_0600_0',
   'properties': {'mean': 490.1700134277344}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-62.119581801721324, -10.215769049745338]},
   'id': 'A20200115_0900_0',
   'properties': {'mean': 489.51800537109375}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-62.119581801721324, -10.215769049745338]},
   'id': 'A20200115_1200_0',
   'properti

In [18]:
# def dated(image, feature):
#     date = ee.Image.date(image)
#     return feature.set({'date': date})

In [19]:
# def dated_colls(imagecoll, featurecoll):
#     for image in imagecoll:
#         for feature in featurecoll:
#             def dated(image, feature):
#                 date = ee.Image.date(image)
#                 return feature.set({'date': date})

In [20]:
def dated(image, featurecoll):
    date = ee.Image.date(image)
    featurecoll = ee.FeatureCollection(featurecoll)
    def datefeat(feature):
        return feature.set({'date': date})
    return featurecoll.map(datefeat)

In [21]:
results = ee.FeatureCollection(ee.FeatureCollection(RootMoist_inst.iterate(dated, hh)))
# results = results.flatten()

In [22]:
# RootMoist_inst.getInfo()

In [23]:
# dated_results = ee.FeatureCollection(ee.List(RootMoist_inst.iterate(date, hh)))

#### Exporting CSV Table

In [24]:
foldername = "FNS"
ExprtDescrptn = input("Filename? ")

export_task = ee.batch.Export.table.toDrive(collection=results, folder=foldername, description=ExprtDescrptn, fileFormat='CSV')
export_task_config = {
    'scale': 27750, 'driveFolder': 'FNS'
}
export_task.start()

# ee.batch.data.startProcessing(export_task, export_task_config)

Filename? anything3


In [25]:
# foldername = "FNS"
# ExprtDescrptn = input("Filename? ")

# export_task = ee.batch.Export.table.toDrive(collection=results, folder=foldername, description=ExprtDescrptn, fileFormat='CSV')
# export_task_config = {
#     'scale': 27750, 'driveFolder': 'FNS'
# }
# export_task.start()

# # ee.batch.data.startProcessing(export_task, export_task_config)

Below: Practice / Tests
<br> code burial site